In [34]:
import pandas as pd
import numpy as np
import json
import requests
import re
import pydash
import time



from secondhandsongs import Api
from collections import OrderedDict
from IPython.display import clear_output
import musicbrainzngs as m


Для разнообразия плейлиста пользователя некоторый интерес представляет добавление в список проигрывания разнообразных исполнений песен, т.н. называемых каверов. Их отличительной особенностью является значительное совпадение текстов песни, но наложенных на видоизмененную музыку. Для поиска оригинала песни или поиска каверов на песню требуется предоставить решение. Исходными данными выступают некоторые метаданные песни и разметка кавер/не кавер. Априорно мы утверждаем, что оригинальная песня - это песня, появившаяся раньше всех. 
Следует отметить, что для создания плейлиста требуется время не более секунды, иначе впечатление от качества работы сервиса будет испорчено. Ввиду большого количества пользователей нагрузка на инфраструктуру должна быть по возможности минимальной. 
Поиск через сравнение текстов.
Показатель сходства текстов является важным признаком сходства песен. При очень высокой степени сходства текстов можно утверждать, что  перед нами связанные песни. С некоторой вероятностью самая первая песня является оригиналом. Не для каждой песни имеются слова песни. С развитием технологий появятся нейросети, которые смогут составлять автоматически тексты к песне.  На данный момент этот вариант сложно реализуем. 
Вторая проблема - проклятие размерности. Имея несколько миллионов треков с закодированными через эмбендинги текстами получаются огромные разреженные матрицы. Для подбора классификации требуется вычисление расстояний(косинусного, левенштейна или другого) между песнями.  Такой процесс возможно производить в фоне, создавая списки каверов к родительскому оригиналу, однако требует предварительного скачивания большой базы текстов, применения токенизаторов для разных языков.
Несколько упрощенной моделью будет создание из текста каждой песни списка 5...15 самых повторяющихся слов длиной более 4 символов. Связанные песни можно определять или прямым сравнением или расстоянием Левенштейна. Это позволит иметь быструю и легковесную модель, инвариантную к языкам. 
Поиск через запросы к информационным сайтам.
Имеется несколько баз, предоставляющих информацию об выпущенных песнях. Зачастую каверы имеют одинаковое название с оригиналом. Получив список песен можно найти оригинал как первую песню из найденного. Это хороший метод, если название песни не менялось. 
Поиск через специализированные сайты. 
Ряд сайтов предоставляют сервис, где перечислены оригинальные песни и их каверы. Плюсом такого решения является высокая точность предсказания, т.к. база корректируется и обновляется сообществом. 
Поиск через чат-гпт. 
В последнее время языковые модели могут отвечать на вопросы, что возможно использовать для ответа на вопрос об оригинальном исполнителе песни. К сожалению проведенное 
Каждый из этих методов имеет плюсы и минусы. Вполне возможно использовать все эти методы вместе, получив ансамбль предсказаний, но потеряв в производительности. 
   


# Первый путь - через secondhandsongs
  
        Сайт secondhand представляет собой базу сообщества, которое собирает информацию про каверы и их оригиналы. Большим плюсом является формат, в котором выдается результат: cоставляется список каверов, выдается id оригинальной песни. Недостатком сервиса является то, что он не поддерживает поиск по ISRC, поэтому однозначно установить правильность найденного результата нельзя. Вместе с тем такую оговорку следуюет добавлять почти ко всем вариантам, т.к.ISCR появился только в середине 80-х, а многие песни происходят из более далеких времен. Название исполнителя и альтернативные названия песни получаем через БД Deezer и БД MusicBrainz. Для работы используется API, который работает следующим образом: посылается ссылка-запрос, а сайт(если запрос верный) в ответ присылает json-файл.
        Результаты запроса аккумулируются в два списка - список названий песен и список названий исполнителя. Это необходимо для того, чтобы найти разные названия одной и той же песни или исполнителя. 

In [2]:
def find_info_by_isrc(isrc, track = None):
    """
    Функция делает запрос на сайт musicbrainz и deezer для поиска
    названия исполнителя и названия песни по ISCR. Т.к. песня может 
    по-разному называться в разных базах. Функция возвращает список 
    как может называться исполнитель и песня.
    """
    # создаем списки для сбора вариаций названий песни и исполнителя
    artist = []
    track_similar = [track]
    print('Делаем запрос в Deezer и MusicBrainz на поиск песни и названия исполнителя')
    # делаем запрос на Deezer и получаем словарь результатов
    dzr_res = requests.get('https://api.deezer.com/2.0/track/isrc:'+isrc).json()
    if 'error' not in dzr_res:
        artist.append( dzr_res.get("artist").get("name") )
        track_similar.append( dzr_res.get("title_short") )
    
    # делаем запрос на Musicbrainz и получаем json
    m.set_useragent("custom_app", "0.01", "sidordima@gmail.com")
    
    try:
        mb_res = m.get_recordings_by_isrc(isrc = isrc)
    except:
        mb_res = None

    # если ответ ненулевой, то выцепляем данные
    # в глубоких словарях поможет библиотека pydash
    if mb_res!=None:
        mb_artist = pydash.get(mb_res, "isrc.recording-list.0.artist-credit.0.artist.name")
        mb_track = pydash.get(mb_res, "isrc.recording-list.0.title")
        artist.append(mb_artist)
        track_similar.append(mb_track)

    # очищаем списки от повторов и None
    artist = list(filter(None, artist))
    artist = list(OrderedDict.fromkeys(artist)) 
    track_similar = list(filter(None, track_similar))
    track_similar = list(OrderedDict.fromkeys(track_similar)) 
    # возвращаем варианты названия песни и артиста 
    return artist,track_similar


После получения названий песни и артиста ищем эту песню на secondhandsong(далее SHS). Такой поиск - задача нетривиальная. Сначала ищется песня по названию, если песня одна - скорее всего это искомая песня. Если песен много, то ищется дополнительно совпадающее название исполнителя. Если ничего не нашлось, то песня ищется по другому названию. И так до перебора всех вариантов. Ввиду ограничения в 10 запросов для бесплатного получения информации поиск не самый быстрый. 

In [3]:
def secondhand_find_song(list_track = None,list_artist = None):
    """
    Запрашиваем результат с сайта secondhand. Для этого ищем id песни запросами 
    из разных названий одной и тоже песни или автора. Возвращаем id с SHS
    """
    # добавим вариант поиска без автора с помощью вставки None в список исполнителей
    list_artist.insert(0,None)
    # активируем режим поиска и обнуляем переменные
    search = True
    i, k = 0, 0
    while search:
        """
        Делаем запрос для получения id. 
        Если one_result=True-значит нашлась только одна песня, это идеальный вариант,
        можно выходить из поиска. Если one_result=False-значит нашлось много песен, 
        требуется уточнение исполнителя. Если is_shs равно None, то пытаемся другое
        название этой же песни найти. 
        """
        print('Идет поиск трека:', list_track[i],", исполнитель: ",list_artist[k])
        id_shs, one_result = SHS_get_id(song=list_track[i], artist=list_artist[k])
        print("Получили такой ids", id_shs, 'and' ,'вариант один',one_result)
        if id_shs != None and one_result==True:
            search = False
            
        elif id_shs != None and one_result==False:
            # если много результатов, то меняем название исполнителя с None 
            k += 1
            # если много одинаковых песен с разными исполнителями
            if k>(len(list_artist)-1):
                return id_shs
        elif id_shs == None:
            # если не нашли песню, то ищем по другому названию
            if i<(len(list_track)-1):
                i += 1
                k = 0
            else:
                return None
        else:
            search = False
            return id_shs
        
        time.sleep(6)
        
    return id_shs

def SHS_get_id(song=None, artist=None):
    """
    Функция делает запрос по названию песни и/или названию исполнителя.
    В ответ получаем результаты поискового запроса
    Извлекаем id первой песни с json-ответа, если он есть. Если нет, то None.
    Если результат один, то вторая переменная принимает значение True
    Если результатов поиска 2 и более, то вторая переменная False
    """
    # делаем запрос с параметрами payload
    payload = {'title': song, 'performer': artist, 'format': 'json'}
    response = requests.get('https://secondhandsongs.com/search/performance', params=payload)
    
    # если есть коннект, то получаем данные
    if response.status_code == 200:
        # Получаем данные из ответа (JSON-данные)
        result = response.json()
    
    # Используем регулярное выражение для извлечения строки из первого результата 
    # id это число после последнего '/'
    if result.get('totalResults') >= 1:
        #берем ссылку на исполнителя 
        uri = result['resultPage'][0]['uri']
        # вытягиваем ид с помощью регулярного выражения
        match = re.search(r"\d+(?=\/?$)", uri )
        if match:
            result_id = match.group(0)
        else:
            print('Не получилось извлечь id')
    else:
        return None, True
    
    if result.get('totalResults')==1:
        return result_id, True
    else:
        return result_id, False

Если на сайте secondhand найдена песня, то можем вывести информацию о песне. 

In [4]:
def SHS_print_info(id=None):
    """
    Функция делает запрос по id песни на secondhand.
    Через спец библиотеку получаем данные, где указано оригинал или нет, список id каверов
    """
    clear_output()
    # инициализируем класс без спец ключа
    shs_api = Api(api_key=None)
    # делаем запрос по id perfomance и получаем ответ
    p = shs_api.get_performance(id)
    
    # выводится полная инфа
    if p.is_original:
        print("Это оригинал песни")
        print("Название песни:", p.title)
        print("Название исполнителя:",p.performer_name)
        print("ИД оригинала песни:", p.originals)
        print("Каверы на эту песню:", p.cover_ids)
        print("Ссылка на secondhand:", p.uri)
    else:
        print("Это кавер-версия песни")
        print("Название песни:", p.title)
        print("Название исполнителя:",p.performer_name)
        print("ИД оригинала песни:", p.originals)
        print("Ссылка на secondhand:", p.uri)
        
        covers = shs_api.get_performance(p.originals[0])  
        print("Найдены каверы на эту песню:", covers.cover_ids)
        
    return None


Весь путь метода через secondhand описывается этой функцией. 

In [5]:
def secondhand_way(input_isrc = None, input_track = None):
    performer, track_names = find_info_by_isrc(isrc=input_isrc, track=input_track)
    print("Исполнители: ",performer)
    print("Названия треков: ", track_names)
    id_song = secondhand_find_song(list_track = track_names,list_artist = performer)
    #clear_output()
    if id_song!=None :
        SHS_print_info(id_song)
    else:
        clear_output()
        print('К сожалению в secondhand нет информации о песне')

In [6]:
#secondhand_way(input_isrc="USWB19600698",input_track='Blackbird')
secondhand_way(input_isrc="RUB861900320",input_track='в городе где нет метро')


К сожалению в secondhand нет информации о песне


====================================
Конец первого способа поиска - через second hand

## Второй путь - через названия песен
Альтернативный путь представляет собой метод, в основе которого лежит следующее допущение: самая старая найденная песня с этим названием является оригиналом, а более свежие являются каверами. Наиболее полной базой является база Deezer через нее будем делать запрос.  

In [7]:
name_track = ''


In [8]:
#song = 'я сошла с ума'
in_isrc =  "RUA110100031"
#in_isrc = "USM951500102"
#in_isrc = "USA370512431"

In [66]:

def find_original(isrc):

    m.set_useragent("custom_app", "0.01", "sidordima@gmail.com")

    try:
        mb_res = m.get_recordings_by_isrc(isrc = isrc)
    except:
        mb_res = None
    print(mb_res)
    # если ответ ненулевой, то выцепляем данные
    # в глубоких словарях поможет библиотека pydash
    if mb_res!=None:
        mb_artist = pydash.get(mb_res, "isrc.recording-list.0.artist-credit.0.artist.name")
        mb_track = pydash.get(mb_res, "isrc.recording-list.0.title")
        

    dzr_resp = requests.get('https://api.deezer.com/2.0/track/isrc:'+isrc).json()
    if 'error' not in dzr_resp:
        dzr_id = pydash.get(dzr_resp, "id")
    time.sleep(0.05)
    dzr_resp = requests.get('https://api.deezer.com/2.0/track/isrc:'+isrc).json()

    #dzr_res = requests.get('https://api.deezer.com/track/'+str(dzr_id)).json()
    #print(json.dumps(dzr_res, ensure_ascii=False, indent=4))

    dzr_res = requests.get('https://api.deezer.com/search/track?q=:'+"blackbird").json()
    print(json.dumps(dzr_res, ensure_ascii=False, indent=4))
    return dzr_res
    


dzr_list = find_original('USM951500102')


def dzr_find_id(id):
    artist = []
    track_similar = []
    
    
    if 'error' not in dzr_res:
        artist.append( dzr_res.get("artist").get("name") )
        track_similar.append( dzr_res.get("title_short") )
       


{'isrc': {'id': 'USM951500102', 'recording-list': [{'id': 'fef5d35b-4a33-45d8-9c79-a74f23f0adbe', 'title': 'Blackbird', 'length': '140958'}, {'id': '02efe477-a61d-4c09-8d78-9d193c74b987', 'title': 'Blackbird', 'length': '140000'}], 'recording-count': 2}}
{
    "data": [
        {
            "id": 116348212,
            "readable": true,
            "title": "Blackbird (Remastered 2009)",
            "title_short": "Blackbird",
            "title_version": "(Remastered 2009)",
            "link": "https://www.deezer.com/track/116348212",
            "duration": 138,
            "rank": 787391,
            "explicit_lyrics": false,
            "explicit_content_lyrics": 0,
            "explicit_content_cover": 0,
            "preview": "https://cdns-preview-9.dzcdn.net/stream/c-940693c0a30e575d6d0e6466e1c80763-12.mp3",
            "md5_image": "f8b236243adae6bc187d27157bc61ae9",
            "artist": {
                "id": 1,
                "name": "The Beatles",
                "link

In [78]:
number = pydash.get(dzr_list, "total")
true_list = []
dzr_dict = {}
for i in range(number):
    track = pydash.get(dzr_list, "data."+str(i)+".title")
    if track == "Blackbird":
        true_list.append(pydash.get(dzr_list, "data."+str(i)+".id"))

print(true_list)
for id in true_list:
    dzr_resp = requests.get('https://api.deezer.com/track/'+str(id)).json()
    if 'error' not in dzr_resp:
        clear_output()
        print(json.dumps(dzr_resp, ensure_ascii=False, indent=4))
        time.sleep(10)
        print(pydash.get(dzr_resp, "data.release_date"))
        dzr_list[id] = pydash.get(dzr_resp, "data.release_date")

#print(dzr_list)
    

{
    "id": 536563392,
    "readable": true,
    "title": "Blackbird",
    "title_short": "Blackbird",
    "isrc": "FR0Q91801709",
    "link": "https://www.deezer.com/track/536563392",
    "share": "https://www.deezer.com/track/536563392?utm_source=deezer&utm_content=track-536563392&utm_term=0_1699024117&utm_medium=web",
    "duration": 277,
    "track_position": 9,
    "disk_number": 1,
    "rank": 341782,
    "release_date": "2018-09-14",
    "explicit_lyrics": false,
    "explicit_content_lyrics": 0,
    "explicit_content_cover": 2,
    "preview": "https://cdns-preview-6.dzcdn.net/stream/c-60fe64fb8beca7d95bdde50fb5d01dd3-6.mp3",
    "bpm": 114.21,
    "gain": -9.6,
    "available_countries": [
        "AE",
        "AF",
        "AG",
        "AI",
        "AL",
        "AM",
        "AN",
        "AO",
        "AQ",
        "AR",
        "AS",
        "AT",
        "AU",
        "AZ",
        "BA",
        "BB",
        "BD",
        "BE",
        "BF",
        "BG",
        "BH",

In [73]:
print(json.dumps(dzr_list, ensure_ascii=False, indent=4))

{
    "data": [
        {
            "id": 116348212,
            "readable": true,
            "title": "Blackbird (Remastered 2009)",
            "title_short": "Blackbird",
            "title_version": "(Remastered 2009)",
            "link": "https://www.deezer.com/track/116348212",
            "duration": 138,
            "rank": 787391,
            "explicit_lyrics": false,
            "explicit_content_lyrics": 0,
            "explicit_content_cover": 0,
            "preview": "https://cdns-preview-9.dzcdn.net/stream/c-940693c0a30e575d6d0e6466e1c80763-12.mp3",
            "md5_image": "f8b236243adae6bc187d27157bc61ae9",
            "artist": {
                "id": 1,
                "name": "The Beatles",
                "link": "https://www.deezer.com/artist/1",
                "picture": "https://api.deezer.com/artist/1/image",
                "picture_small": "https://e-cdns-images.dzcdn.net/images/artist/fe9eb4463ea87452e84ed97e0c86b878/56x56-000000-80-0-0.jpg",
          

Непонятный вывод - почему нет имени артиста и год указан как 2006? Ведь песня известная Тату и год 2000 должен быть. Попробуем отдельно к каждой базе обратиться. 

import re
# обращаемся к musicbrain
musicbrain = linking.MusicBrainzAlign(isrc = in_isrc)
mb_out = musicbrain._search_by_isrc()
#очень длинный вывод в json
if mb_out!=None:
    mb_artist = mb_out["isrc"]["recording-list"][0]["artist-credit"][0]['artist']['name']
    str_json = json.dumps(mb_out)
    mb_year = min( re.findall(r"(\d{4}\-\d{2}\-\d{2})", str_json) )
else:
    mb_year, mb_artist = None, None

print(mb_artist)
print(mb_year)

In [ ]:
url = 'https://api.deezer.com/2.0/track/isrc:RUB861900320'
response = requests.get(url)

# если все хорошо, то бует статус 200
if response.status_code == 200:
    # Получаем данные из ответа (например, JSON-данные)
    json_data = response.json()
    
else:
    # В случае ошибки выводим сообщение
    print(f"Ошибка: {response.status_code}")



formatted_json = json.dumps( json_data, ensure_ascii=False, indent=4)

print(json.dumps( json_data, ensure_ascii=False, indent=4))

print(json_data["artist"]["name"])

def DZR_get_info(isrc=None):
    """
    Делаем запрос по ISRC на сайт Deezer для поиска инфы об исполнителе
    """
    # если прислали ненеулевую переменную, то формируем и делаем запрос
    if isrc!=None:
        dzr_url = 'https://api.deezer.com/2.0/track/isrc:'+isrc
        response = requests.get(dzr_url)
    else:
        return None
    
    # запрашиваем данные
    if response.status_code == 200:
    # Получаем данные из ответа (JSON-данные)
        json_data = response.json()
    else:
        # В случае ошибки выводим сообщение
        print(f"Ошибка: {response.status_code}")
        return None
    
    return json_data
    